In [1]:
# import packages
import tensorflow as tf
import numpy as np

print(tf.__version__)

/Users/nityansuman/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


ImportError: cannot import name 'abs'

In [2]:
# constant is used to hold fixed numeric values
node1 = tf.constant(3.0, tf.float32) # nodes are nothing but tensors
node2 = tf.constant(4.5) # implicitly applied: float332

In [3]:
print(node1, node2) # view the nodes

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


In [4]:
# create a session object to evaluate nodes
sess = tf.Session()

In [5]:
# evaluate the node by running it using the session object
sess.run(node1)

3.0

In [6]:
# you can evaluate multiple nodes at once
sess.run([node1, node2])

[3.0, 4.5]

In [7]:
# performing basic operation on the nodes
sess.run(tf.add(node1, node2)) # add

7.5

In [8]:
sess.run(tf.multiply(node1, node2)) # multiply

13.5

In [9]:
sess.run(tf.subtract(node1, node2))

-1.5

In [10]:
node3 = tf.add(node1, node2) # creating new node
node3 # no output: lazy evaluation

<tf.Tensor 'Add_1:0' shape=() dtype=float32>

In [11]:
sess.run(node3) # evaluate the node

7.5

In [12]:
# placeholders are used to hold variable input
a = tf.placeholder(tf.int32) # define the data type of the input
b = tf.placeholder(tf.int32)

In [13]:
adder_node = tf.add(a, b) # create new node that perfroms some basic operation like add

In [14]:
# create and evaluate the graph containg the 3 nodes
# feed dict is used to pass the values to the placeholders
sess.run(adder_node, feed_dict={a: 2, b: 4})

6

In [15]:
# multiple inputs can be passed using the feed dict as well
sess.run(adder_node, feed_dict={a:[1, 2], b:[3, 10]})

array([ 4, 12], dtype=int32)

In [16]:
result = sess.run(adder_node, feed_dict={a:[2, 3, 4], b:[4, 5, 6]}) # save the result into another node

In [17]:
result.shape # shape of the resulting array

(3,)

In [18]:
print(result[0], result[1], result[2]) # access individual results

6 8 10


Constants are nothing but fixed numeric values.

Placeholder is used to handle values which wont change in time like inputs.

Variables are used to store trainable data which needs to be changed in between.

In [19]:
# lets implement a simple linear regression model
weight = tf.Variable([1.0], tf.float32) # set intial weight as 1.0
bias = tf.Variable([1.0], tf.float32) # set initial weight as 1.0
x = tf.placeholder(tf.float32)

# linear regression model
model = weight * x + bias

# initialize all variables: necessary when using variables
sess.run(tf.global_variables_initializer())

# now run the model
sess.run(model, feed_dict={x:[1, 2, 3, 4, 5]})

array([2., 3., 4., 5., 6.], dtype=float32)

In [20]:
# create placeholder for predicted value
y = tf.placeholder(tf.float32)

# create a error metric
error_measure = tf.squared_difference(model, y)

# create a loss function
loss_function = tf.reduce_mean(error_measure)

# now run to evaluate the loss function: finding loss with current configuration
print(sess.run(loss_function, feed_dict={x:[1, 2, 3, 4], y:[-1, -2, -3, -4]}))

# let us do some prediction to see how close it can get
# we take the testing data same as train data assuming it will perform way better as it knows from training
print(sess.run(model, feed_dict={x:[1, 2, 3, 4], y:[-1, -2, -3, -4]}))

41.0
[2. 3. 4. 5.]


As we can see that prediciton is way far from the original output expected. Since the error is 41.0
We need to tune our model i.e., weights and baises to minimize the loss and thus improve prediction accuracy.

## Linear Regression in Tensorflow Using Gradient Descent Optimizer

In [21]:
# first set some initial values for weight and bias: same as before
weight = tf.Variable([1.0], tf.float32)
bias = tf.Variable([1.0], tf.float32)

# define a placeholder for the input feature
x = tf.placeholder(tf.float32)

# define a placeholder for the output values
y = tf.placeholder(tf.float32)

# define the model
linear_model = tf.add(tf.multiply(weight, x), bias)

# define the loss function
loss_function = tf.reduce_mean(tf.squared_difference(linear_model, y)) # minimize the mean squared difference

In [22]:
# use an optimizer to optimize the weights and bias
# use learning rate of 0.01: we can also use a variable learning rate in future
alpha = tf.constant(0.01, tf.float32)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=alpha)

In [23]:
# use the optimizer to minimise the loss function
trainer = optimizer.minimize(loss_function)

In [25]:
# run the optimizer for 1000 iterations, check values for both weight and bias after every 100 iteration
sess.run(tf.global_variables_initializer()) # reset values to incorrect defaults.

# training data
x_train = [1, 2, 3, 4, 5]
y_train = [-1, -2, -3, -4, -5]

# validation data
x_val = [1, 4, 5]
y_val = [-1, -4, -5]

# testing data
x_test = [10, 30, 1, 444]
y_test = [-10, -30, -1, -444]

for i in range(1000):
    sess.run(trainer, {x:x_train, y:y_train})
    if i % 100 == 0:
        print("weights and baias:", sess.run([weight, bias]))
        print("current error:", sess.run(loss_function, feed_dict={x:x_val, y:y_val}))
        print()
    else:
        continue
# check for the new weights and bias
print(sess.run([weight, bias]))

# now run the model and see the prediction
# pass the testing data and not the training data again
print(sess.run(linear_model, feed_dict={x:x_test})) # expected values are in y_test

weights and baias: [array([0.5], dtype=float32), array([0.86], dtype=float32)]
current error: 40.839603

weights and baias: [array([-1.0815015], dtype=float32), array([0.29424587], dtype=float32)]
current error: 0.019699028

weights and baias: [array([-1.0580875], dtype=float32), array([0.20971443], dtype=float32)]
current error: 0.010006421

weights and baias: [array([-1.0414002], dtype=float32), array([0.14946745], dtype=float32)]
current error: 0.005082965

weights and baias: [array([-1.0295066], dtype=float32), array([0.10652825], dtype=float32)]
current error: 0.0025819768

weights and baias: [array([-1.02103], dtype=float32), array([0.07592462], dtype=float32)]
current error: 0.0013115708

weights and baias: [array([-1.0149884], dtype=float32), array([0.0541128], dtype=float32)]
current error: 0.0006662289

weights and baias: [array([-1.0106827], dtype=float32), array([0.03856737], dtype=float32)]
current error: 0.0003384288

weights and baias: [array([-1.0076135], dtype=float32)

As we can see that we didnt get the exact value but its very close and error has been reduced drastically.